In [8]:
import pandas as pd
import numpy as nmp
import requests
import os
import json

In [5]:
noaa_api = os.getenv('noaa_api')

## NOAA API 

In [11]:
root = 'https://www.ncei.noaa.gov/cdo-web/api/v2'
endpoint = f'/locations'

In [17]:
parameters = {'format': 'json',
            'offset': 85,
            'limit': 50}

In [20]:
headers = {'token': noaa_api}

In [21]:
r = requests.get(root + endpoint,
                params = parameters,
                headers = headers)

In [22]:
r

<Response [200]>